# 1) GGUF Model Loading1:
- ctransformers 
OllamaLLm and HuggingFaceEmbedding: mistralai/Mistral-7B-Instruct-v04
gguf model = 

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [10]:
from langchain_community.document_loaders import PyPDFLoader
file_path = ("Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


15

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

## Create vector store
    stores embeddings of Documents

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

In [14]:
## Locally loading embedding
### Still pending --------------------------->>>>>>>>>>>>>>
# new_embeddings = HuggingFaceEmbeddings(cache_folder= r"C:\Users\30078206\Downloads\sentence-transformersall-mpnet-base-v2")


In [15]:
## Not local
new_embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=new_embeddings)
vectorstore

c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Create retriever

In [16]:
retriever = vectorstore.as_retriever(search_type="similarity")

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### LLM Used:

## Quantized model , using LLama.cpp 

### How to load gguf model in Python code:
- using ctransformers
- using llama-cpp-python 

In [18]:
pip -q install ctransformers

Note: you may need to restart the kernel to use updated packages.


In [19]:
from ctransformers import AutoModelForCausalLM

In [20]:
model_path = r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\mistral_7B_v0.3"

In [21]:
### did not work
not_working_llm =AutoModelForCausalLM.from_pretrained(
    model_path,
    model_file = "Mistral-7B-Instruct-v0.3.Q4_K_M.gguf",
    gpu_layers = 0
)

In [22]:
from langchain_community.llms import CTransformers

In [23]:
model_path = r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\mistral_7B_v0.3"
llm = CTransformers(model= model_path, config={'max_new_tokens': 600,
                              'temperature': 0.01,
                              'context_length': 700})


In [24]:
llm.invoke("modern art is")

' a term that encompasses a wide range of artistic styles and movements that emerged in the late 19th and early 20th centuries. It is characterized by a rejection of traditional forms and techniques, an emphasis on individual expression, and a focus on the artistic process rather than the finished product.\n\nSome of the most famous modern art movements include:\n\n* Impressionism (late 19th century): This movement was characterized by a focus on capturing the fleeting effects of light and color in outdoor scenes. Some famous impressionist artists include Claude Monet, Pierre-Auguste Renoir, and Edgar Degas.\n* Post-Impressionism (late 19th century): This movement built upon the techniques of impressionism but pushed them further, often using bold colors and distorted forms to express emotion. Some famous post-impressionist artists include Vincent van Gogh, Paul Cézanne, and Georges Seurat.\n* Fauvism (early 20th century): This movement was characterized by the use of bright, non-natur

### Creating Prompt

In [25]:
from langchain.prompts import PromptTemplate

In [26]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


### New Terms:
- Multiqueryretriever: 
    - automates process of tuning
    - to generate multiple queries from different perspective 
    - for each query- returns relevant documents,, takes union across all 
    - Overcomes the limitation of distance based retrieval


In [27]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate

In [28]:
retriever = MultiQueryRetriever.from_llm(
    retriever, 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [29]:
retriever

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000027AEE494B90>), llm_chain=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five\n    different versions of the given user question to retrieve relevant documents from\n    a vector database. By generating multiple perspectives on the user question, your\n    goal is to help the user overcome some of the limitations of the distance-based\n    similarity search. Provide these alternative questions separated by newlines.\n    Original question: {question}')
| CTransformers(client=<ctransformers.llm.LLM object at 0x0000027AF2671C10>, model='D:\\OneDrive - Adani\\Desktop\\LEARNING_FOLDER\\_Kolkata_2024\\1_LLM\\local_downloaded_models\\mistral_7B_v0.3', config={'max_new_tokens': 600, 'temperature': 0.01, 'context_length': 700})
| LineListOutputParser())

In [30]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [31]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x0000027AF2671C10>, model='D:\\OneDrive - Adani\\Desktop\\LEARNING_FOLDER\\_Kolkata_2024\\1_LLM\\local_downloaded_models\\mistral_7B_v0.3', config={'max_new_tokens': 600, 'temperature': 0.01, 'context_length': 700})

In [34]:
## Plain QA Chain

In [35]:
from langchain.chains.question_answering import load_qa_chain

In [36]:
stuff_chain = load_qa_chain(llm, chain_type="stuff")

In [37]:
stuff_chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [38]:
new_retriever = vectorstore.as_retriever(search_type="similarity")

In [ ]:
stuff_chain.invoke(inpu)

In [32]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
questions = chain.invoke("Give a summary of the document given")

INFO:langchain.retrievers.multi_query:Generated queries: ['its title "The Impact of Climate Change on Agriculture".', '', '1. Summarize the content of the article titled "Climate Change Effects on Agriculture".', '2. What does the document titled "Impact of Global Warming on Farming" discuss?', '3. Provide a brief overview of the paper entitled "Agriculture and Climate Change: A Comprehensive Review".', '4. Can you summarize the main points of the article with the title "Climate Change\'s Influence on Agricultural Practices"?', '5. What is the focus of the document titled "The Role of Climate Change in Shaping Modern Agriculture"?']


In [56]:
print(questions)

What is: What is:
What type:
What type [1 = { "Leave Policy for this version HRL: What is: What does not included here...





-
1 about Privile 'Leave Policy on leave policy regarding entitle Leave_context are:
1?











What is: What is: What type are based only regarding Entitself.







What is "Leave Policy regarding the entitle, with respect to answer the definition of Leave Policy:
How many ways to you provided here, about types and answer the definition?
What is: What is: What is: What is: What is: What is as follows:
What type multiple choice 1? What does not answered as needed: "Leave_id 1: What is: What is: What is: What is: What is:
-
An employee earns provided?

What is: What is: What is: What is: What is: What is: What is: What is: What is: What is: What is: What is "Leave Policy regarding leave policy?
What type of the entitself.


In [ ]:
import re

In [ ]:
answer =re.sub(r'[0-9]','',questions)
question_list = answer.split('\n')
question_list

In [ ]:
chain.invoke(question_list[0])

In [ ]:
''' Taking too much time'''
# answer_list = []
# for question in question_list:
#     ans = chain.invoke(question)
#     answer_list.append(ans)

In [ ]:
print("ANSWERS-----")
# print(answer_list)

In [ ]:
question_bank = [
    "Give me a summary of the document",
    "Tell me all about the leaves",
    "Make a list of questions from the document"
]

--------------------------------------------(2)--------------------------------------------

In [ ]:
## huggingfaceEmbedding issue

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# llm_embedding = HuggingFaceEmbeddings(
#     model_name = "sentence-transformers/all-mpnet-base-v2")


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs)

In [26]:
from langchain_huggingface import HuggingFaceEndpoint

In [32]:
model_repo = "mistralai/Mistral-7B-Instruct-v0.2"
google_llm =HuggingFaceEndpoint(
    repo_id= model_repo,
    max_new_tokens=250
)

In [34]:
google_llm

HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=250, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>)

In [40]:
from langchain_community.llms import CTransformers

In [37]:
model_path = r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\mistral_7B_v0.3"
llm = CTransformers(model= model_path)


In [39]:
llm.invoke("modern art is")

' a term that encompasses a wide range of artistic styles and movements that emerged in the late 19th and early 20th centuries, characterized by a rejection of traditional rules and an emphasis on creativity, innovation, and individual expression.\n\nThe roots of modern art can be traced back to the Industrial Revolution, which brought about significant changes in society and culture, including rapid urbanization, increased mobility, and new technologies. These changes led to a sense of disillusionment with traditional values and beliefs, as well as a desire for experimentation and innovation in all areas of life, including art.\n\nSome key figures in the early development of modern art include Vincent van Gogh, who pioneered the use of bold colors and expressive brushstrokes in his paintings; Paul Cézanne, who is often credited with laying the groundwork for Cubism through his emphasis on geometric forms and abstract shapes; and Edvard Munch, whose work explored themes of anxiety, des